In [213]:
import pandas as pd
import numpy as np


In [214]:


df = pd.read_excel("Hw1-words.xlsx", sheet_name='Proto-words')  # Replace 'Sheet1' with the name of the sheet you want to read
df


,father,mother,one,eight,knee,nose,tongue,dog,egg,smoke,snow,to burn,to bite,to sing,to spit,dry
0,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*mura,*śil'ke,*kuśka
1,*ićä,*emä,*ikte,*kakteksan,*polwe,*nenä,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa
2,*ićä,*emä,*egy,*ńalɜ,*térd,*orr,*nyelv,*ämpɜ,*tojás,*pičɜ,*hó,*pilɜ,*harap,*énekel,*köp,*ϑasɜ
3,*ićä,*emä,*ikte,*ńalɜ,*sāns,*nere,*kele,*ämpɜ,*muna,*pičɜ,*tūjt,*pilɜ,*pure,*mura,*śil'ke,*ϑasɜ
4,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*jylme,*pene,*muna,*šikš,*lume,*äŋɜ,*pure,*mura,*śil'ke,*kuśka
5,*ićä,*emä,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa


In [215]:

df.columns



Index(['father', 'mother', 'one', 'eight', 'knee', 'nose', 'tongue', 'dog',
       'egg', 'smoke', 'snow', 'to burn', 'to bite', 'to sing', 'to spit',
       'dry'],
      dtype='object')

In [216]:
word_list = []
for i, column in enumerate(df.columns):
    # print(i, column)
    word_list.append(df[column].tolist())
# word_list = word_list.tolist()
word_list



[['*ićä', '*ićä', '*ićä', '*ićä', '*ićä', '*ićä'],
 ['*ava', '*emä', '*emä', '*emä', '*ava', '*emä'],
 ['*ikte', '*ikte', '*egy', '*ikte', '*ikte', '*ikte'],
 ['*kakteksan', '*kakteksan', '*ńalɜ', '*ńalɜ', '*kakteksan', '*kakteksan'],
 ['*polwe', '*polwe', '*térd', '*sāns', '*polwe', '*polwe'],
 ['*nere', '*nenä', '*orr', '*nere', '*nere', '*nere'],
 ['*kele', '*kele', '*nyelv', '*kele', '*jylme', '*kele'],
 ['*pene', '*pene', '*ämpɜ', '*ämpɜ', '*pene', '*pene'],
 ['*muna', '*muna', '*tojás', '*muna', '*muna', '*muna'],
 ['*sawe', '*sawe', '*pičɜ', '*pičɜ', '*šikš', '*sawe'],
 ['*lume', '*lume', '*hó', '*tūjt', '*lume', '*lume'],
 ['*korpe', '*korpe', '*pilɜ   ', '*pilɜ   ', '*äŋɜ', '*korpe'],
 ['*pure', '*pure', '*harap', '*pure', '*pure', '*pure'],
 ['*mura', '*laulu', '*énekel', '*mura', '*mura', '*laulu'],
 ["*śil'ke", "*śil'ke", '*köp', "*śil'ke", "*śil'ke", "*śil'ke"],
 ['*kuśka', '*kujwa', '*ϑasɜ', '*ϑasɜ', '*kuśka', '*kujwa']]

In [217]:
values_list_flat = [val for sublist in df.values.tolist() for val in sublist]
uniques = np.unique(values_list_flat)
print(uniques)
print (len(uniques))



['*ava' '*egy' '*emä' '*harap' '*hó' '*ikte' '*ićä' '*jylme' '*kakteksan'
 '*kele' '*korpe' '*kujwa' '*kuśka' '*köp' '*laulu' '*lume' '*muna'
 '*mura' '*nenä' '*nere' '*nyelv' '*orr' '*pene' '*pilɜ   ' '*pičɜ'
 '*polwe' '*pure' '*sawe' '*sāns' '*tojás' '*térd' '*tūjt' '*ämpɜ' '*äŋɜ'
 '*énekel' '*ńalɜ' "*śil'ke" '*šikš' '*ϑasɜ']
39


In [218]:
keys = list('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklm')
len(keys)


proto_dict = dict(zip(uniques, keys))

df.replace(proto_dict, inplace=True)




In [219]:
matrix_size = df.shape[0]
NameIndexed = []
for i in range(matrix_size):
    NameIndexed.append("S"+str(i+1))
print(NameIndexed)

['S1', 'S2', 'S3', 'S4', 'S5', 'S6']


In [220]:
distance_matrix = pd.DataFrame(index=range(df.shape[0]), columns=range(df.shape[0]))
distance_matrix = pd.DataFrame(distance_matrix, index=NameIndexed)

In [253]:
min_value_location = []
min_value = 999999
for i in range (matrix_size):
    for j in range (matrix_size-i,0,-1):
        col=matrix_size-j
        row =i
        row1 = df.iloc[row]
        row2 = df.iloc[col]
        distance = (row1 != row2).sum()
        if min_value>distance and distance!=0:
            min_value = distance
            min_value_location = (row,col)        
#         print(f"Distance between row {row} and {col}", "Distance =", distance)
        distance_matrix.iloc[row,col]=distance


### <u>**Distance matrix from Proto words**</u>

In [252]:
distance_matrix

,S1,S2,S3,S4,S5,S6
S1,0,4,15,8,3,3
S2,NaN,0,14,9,7,1
S3,NaN,NaN,0,9,15,14
S4,NaN,NaN,NaN,0,9,8
S5,NaN,NaN,NaN,NaN,0,6
S6,NaN,NaN,NaN,NaN,NaN,0


### Renaming columns as S1..Sn & finding the row/col index of the minimum distance

In [222]:
distance_matrix.columns = NameIndexed
distance_matrix.shape
lowerIndex, upperIndex = min_value_location

In [235]:
newRowColName = f"S{lowerIndex+1}{upperIndex+1}"

### Creating a copy of distance matrix for calculation of 2nd step and removing the rows and column associated with minimum row/col distance.

In [242]:
distance_matrix_l1 = distance_matrix.copy()

In [243]:
distance_matrix_l1.drop(distance_matrix_l1.columns[upperIndex], axis=1, inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.columns[lowerIndex], axis=1, inplace=True)

distance_matrix_l1.drop(distance_matrix_l1.index[upperIndex], inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.index[lowerIndex], inplace=True)

In [245]:
newRowColName

'S26'

In [246]:
# distance_matrix_l1[newRowColName]=np.nan

In [251]:
# distance_matrix_l1

In [248]:

rowCombo  = distance_matrix_l1.iloc[0]
rowCombo.iloc[:] = np.nan

In [249]:
distance_matrix_l1.insert(0, f"S{lowerIndex+1}{upperIndex+1}", rowCombo)

distance_matrix_l1.rename(index={distance_matrix_l1.index[0]: f"S{lowerIndex+1}{upperIndex+1}"}, inplace=True)
# distance_matrix_l1.index[0] = f"S{lowerIndex}{upperIndex}"

distance_matrix_l1

,S26,S1,S3,S4,S5
S26,NaN,NaN,NaN,NaN,NaN
S3,NaN,NaN,0,9,15
S4,NaN,NaN,NaN,0,9
S5,NaN,NaN,NaN,NaN,0


In [226]:
dfnew = pd.DataFrame(distance_matrix_l1.iloc[2], index=[0])
dfnew
distance_matrix_l1.shape

(4, 5)

In [ ]:
new

In [227]:
new_distance_matrix = pd.DataFrame()
drop_index = max(min_value_location)
minIndex = min(min_value_location)



In [228]:
drop_index

distance_matrix_l1 = distance_matrix.copy()




In [232]:
distance_matrix_l1

,S1,S2,S3,S4,S5,S6
S1,0,4,15,8,3,3
S2,NaN,0,14,9,7,1
S3,NaN,NaN,0,9,15,14
S4,NaN,NaN,NaN,0,9,8
S5,NaN,NaN,NaN,NaN,0,6
S6,NaN,NaN,NaN,NaN,NaN,0


In [231]:
distance_matrix_l1.drop(distance_matrix_l1.columns[min_value_location], axis=1, inplace=True)

distance_matrix_l1.drop(distance_matrix_l1.index[drop_index], inplace=True)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
distance_matrix_l1

distance_matrix_drop1 = distance_matrix.drop(distance_matrix.columns[drop_index], axis=1)

NameIndexed

distance_matrix.columns = NameIndexed
distance_matrix = pd.DataFrame(distance_matrix, index=NameIndexed)
